In [2]:
import numpy as np
from pathlib import Path
from helpers import inverse_min_max_scaler
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pyvista as pv


In [3]:
import sys
sys.path.append(str(Path().cwd().parent.parent))
from ComsolClasses.comsol_classes import COMSOL_VTU
from ComsolClasses.helper import calculate_normal

In [4]:
ROOT = Path().cwd().parent / "Snapshots" / "01" / "BasisFunctions"
ROOT.exists()

True

### Import

In [5]:
min_max = np.load(ROOT / "min_max.npy")
basis_functions = np.load(ROOT / "basis_fts_matrix.npy")

In [6]:
basis_functions_scaled = inverse_min_max_scaler(basis_functions, min_max[0], min_max[1])

In [7]:
comsol_data = COMSOL_VTU("/Users/thomassimader/Documents/NIRB/Snapshots/01/Training/Training_000.vtu")

In [8]:
normal = calculate_normal(60, 90)
type(normal)

tuple

### Plot

In [25]:
from turtle import title


n_cols = 2  # number of columns you want
n_rows = int(np.ceil(basis_functions.shape[0] / n_cols))  # calculate required number of rows

plotter = pv.Plotter(shape=(n_rows, n_cols))
counter = 0
for i in range(n_rows):
    for j in range(n_cols):
        plotter.subplot(i, j)
        field_name = f"basis_function{i}"
        if counter >= basis_functions.shape[0]:
            break
        comsol_data.mesh.point_data[field_name] = basis_functions_scaled[counter, :]
        clipped = comsol_data.mesh.clip(normal=-np.array(normal), origin = comsol_data.mesh.center)
        
        plotter.add_mesh(clipped, scalars=field_name,
                            cmap='jet',
                            scalar_bar_args={'title': f'Temperature [K] ({counter})',
                                            'label_font_size': 10,
                                            'title_font_size': 8,})
        counter += 1
    
plotter.show()

Widget(value='<iframe src="http://localhost:56492/index.html?ui=P_0x6a81d2e10_15&reconnect=auto" class="pyvist…